### 문제 2단계 체인 만들기 - 영화 추천 시스템 
문제 설명
사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.
요구사항
1단계 체인: 장르를 입력받아 영화 1편 추천
2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
ChatPromptTemplate 사용 
: “system”  과 “human” 메시지를 지정합니다.
두 체인을 LCEL로 연결
각 단계의 결과를 모두 출력하여 과정 확인
구현 조건
입력: 영화 장르 (예: "액션", "로맨스", "공포")
1단계 출력: 추천 영화 제목과 간단한 설명
2단계 출력: 영화 줄거리 3줄 요약


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 그냥 사람이야.") , 
     ("human", "영화 하나만 추천해줘") ]
)

# Step 1: 사용자가 입력한 재료에 따른 요리 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 추천 영화 제목 1개와 그 영화에 대한 간단한 설명")

# Step 2: 추천된 요리의 재료와 레시피 설명명
prompt2 = ChatPromptTemplate.from_template("{storyline}  영화 줄거리 3줄로 요약")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

parser = StrOutputParser()

# 장르
genre_input = "공포"

full_chain = (
    prompt1
    | llm
    | parser
    # 1단계 결과(movie_info)를 {"storyline": movie_info} 형태로 래핑
    | RunnableLambda(lambda movie_info: {"storyline": movie_info})
    | prompt2
    | llm
    | parser
)


# 한 번에 실행하기
result = full_chain.invoke({"genre": "공포"})
print(result)

영화 '헤리티지'의 줄거리 3줄 요약은 다음과 같습니다.

1.  한 가족이 조상들의 유산과 오래된 저택을 물려받습니다.
2.  이 저택에서 기이한 사건들이 벌어지기 시작하고, 가족들은 오래된 저택의 어두운 과거를 탐구하게 됩니다.
3.  이 과정에서 가족의 비밀과 오래된 저택의 비밀들이 드러나고, 가족들은 공포에 직면하게 됩니다.
